# **Santander Dev Week 2023 (ETL com Python)**

## **Contexto** 

Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada.

Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

## **O que será feito?**

Para este desafio criaremos um processo de ETL (Extract, Transform, Load).

* Na primeira etapa (**Extract**) utilizaremos a API ('https://sdw-2023-prd.up.railway.app') disponibilizada no desafio para carregar todos os usuários disponíveis. Por se tratar de uma apresentação do projetos, selecionaremos apenas três clientes para aplicarmos.

* Na fase **Transform** utilizaremos a API da OpenAI para gerar frases personalizadas sobre investimentos para cada cliente.

* Depois de carregar os clientes e criar as frases personalizadas, carregaremos os dados atualizados novamente utilizando a API.



## **E**xtract

Nessa primeira etapa, vamos extrair os dados dos cliente disponibilizados através do API criado para este estudo.

In [1]:
#URL API
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

In [2]:
# Importação bibliotecas
import requests
import json
import pandas as pd

In [3]:
#Solicitar os dados de todos os usuarios
response = requests.get(f'{sdw2023_api_url}/users') 

#Verificar se a solicitação deu certo e imprimir a quantidade de registros importados
#caso ocorra erro, retornar o código de retorno do request
if response.status_code == 200:
    data = response.json()
    num_regisros = len(data)
    print(f'Número de registros no arquivo JSON: {num_regisros}')
else:
    print(f'Falha na solicitação HTTP. Código de status: {response.status_code}')

Número de registros no arquivo JSON: 541


Apenas para demonstração deste projeto, vamos utilizar os cinco últimos registros do arquivo Json:

In [4]:
#Extraindo os 5 últimos registros
three_last_records = data[-3:]
print(json.dumps(three_last_records, indent=2))

[
  {
    "id": 611,
    "name": "fernanda",
    "account": {
      "id": 622,
      "number": "000011",
      "agency": "0001",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 585,
      "number": "0000000000001111",
      "limit": 0.0
    },
    "features": [
      {
        "id": 224,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 1819,
        "icon": "string",
        "description": "string"
      }
    ]
  },
  {
    "id": 612,
    "name": "ana",
    "account": {
      "id": 623,
      "number": "000022",
      "agency": "0002",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 586,
      "number": "0000000000002222",
      "limit": 0.0
    },
    "features": [
      {
        "id": 225,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 1820,
        "icon": "string",
        "description": "string"
      }
    

## **T**ransform

Nesta etapa vamos gerar uma frase personalizada sobre investimos para cada usuário selecionado na etapa anterior.

Utilizaremos a API da OpenAI para utilizar o chatGPT para gerar as frases.

In [5]:
!pip install openai

In [6]:
#Importando OS para carregar chave da api do OPENAI salvo em variável do sistema
import os
import openai

In [7]:
#Carregando API KEY
api_key = os.environ.get("OpenAI_Key")
openai.api_key = api_key

In [8]:
#Definindo a função para geração das frases
def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em investimentos bancário com enfase em marketing digital."
          },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres). Use #SantanderInvestimentos"
          }
        ]
      )
    return completion.choices[0].message.content.strip('\"')

In [9]:
for user in three_last_records:
    news = generate_ai_news(user)
    print(news)
    user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Fernanda, invista no seu futuro com segurança e rentabilidade. Conte com o #SantanderInvestimentos!
Invista no seu futuro com #SantanderInvestimentos!
Invista com segurança e crescimento #SantanderInvestimentos


## **L**oad

Após utilizarmos as APIs para carregar os dados e criar frases sobre investimentos utilizando o ChatGPT, esta na hora de carregar as alterações em nosso banco de dados.

Para isso, utilizaremos a API para fazer um update nos usuários com nossas alterações.

In [10]:
#criando a função para atualizar os dados utilizando a API SDW
def update_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

In [11]:
#iterando sobre os usuários selecionados e fazendo a atualização
for user in three_last_records:
    response = update_user(user)
    if response is True:
        print(f'Usuário {user["name"]} atualizado com sucesso.')
    else:
        print(f'Problemas ao atualizar o usuário{user["name"]}')

Usuário fernanda atualizado com sucesso.
Usuário ana atualizado com sucesso.
Usuário carlos atualizado com sucesso.
